In [1088]:
import sys
import glob, os, os.path
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
#!{sys.executable} -m pip install pillow

In [1089]:
import docx
import pandas as pd
from docx.shared import Pt
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL
from docx.enum.section import WD_SECTION
from docx.shared import Cm, Inches

import barcode
from barcode import UPCA 
from barcode.writer import ImageWriter 

In [1090]:
# Disables writing under barcode image
barcode.base.Barcode.default_writer_options['write_text'] = False

# Generates barcode and saves into folder
def generate_barcode(number):
    upc = str(number)

    # Important to specify UPC type
    my_code = UPCA(upc, writer=ImageWriter()) 
    my_code.save('Barcodes/'+ upc)

In [1091]:
#Sets Margins for each table in the document
def set_section_margins(doc):
    sections = doc.sections
    for section in sections:
        section.top_margin = Inches(0.5)  # Set top margin to 1 inch
        section.bottom_margin = Inches(0.42)  # Set bottom margin to 1 inch
        section.left_margin = Inches(0.56)  # Set left margin to 1.25 inches
        section.right_margin = Inches(0.31)
    

In [1092]:
#Sets dimensions for each printable section in table

def set_table_dim(table):
    for row in table.rows:
        row.height = Inches(1.245)
    index = 1
    for column in table.columns:
        for cell in column.cells:
            if index%2==0:
                cell.width = Inches(0.2)
            else:
                cell.width = Inches(1.75)
        index+=1

In [1093]:
# Populates table with data available in excel document
def populate_table(table,df):
    
    data_index = 0
    max_index = df.shape[0]
    counter = 1
    print(len(table.rows))
    for row in table.rows:
        for cell in row.cells:
            shading_elm_1 = parse_xml(r'<w:shd {} w:fill="FF7F7F"/>'.format(nsdecls('w')))
            if counter%2!=0 and data_index < max_index:
                name = df.loc[data_index,'Description1']
                price = df.loc[data_index,'Price']
                barcode = df.loc[data_index,'Barcode Lookup']
                cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
                p1 = cell.paragraphs[0]
                p1.alignment = WD_TABLE_ALIGNMENT.CENTER
                run = p1.add_run()
                generate_barcode(barcode)
                run.add_picture('Barcodes/'+ str(barcode)+".png"
                                ,width=Inches(1.60))
                p2 = cell.add_paragraph(name + "\n" + "$" + ('%.2f' % price))
                p2.alignment = WD_TABLE_ALIGNMENT.CENTER
    
                data_index +=1
                cell._tc.get_or_add_tcPr().append(shading_elm_1)
            if counter == 7:
                counter = 0
            counter +=1
    
            
    #row = table.rows[0].cells
    #row[0].vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    #p1 = row[0].paragraphs[0]
    #run = p1.add_run()
    #run.add_picture(image,width=Inches(1.60))
    #p2 = row[0].add_paragraph(name + "\n" + "$" + str(price))
    #p2.alignment = WD_TABLE_ALIGNMENT.CENTER
    

In [1094]:
# Creates each table in document
def create_table(doc,df):
    table = doc.add_table(rows=8,cols=7)
    table.alignment = WD_TABLE_ALIGNMENT.LEFT
    table.style = "Table Grid"
    table.autofit = False
    set_table_dim(table)
    populate_table(table,df)
    

In [1095]:
# Separates dataframe into 32 equal sections for each of the pages.
def list_df(df):
    length = df.shape[0]
    i = 0
    ls = []
    while i < length:
        try:
            temp_df = df.loc[i:i+31]
            ls.append(temp_df.reset_index(drop=True))
        except:
            temp_df = df.loc[i:,:]
            ls.append(temp_df.reset_index(drop=True))
        i +=32
    return ls

In [1096]:
df = pd.read_excel('data.xlsx')
df = df.loc[:,['Barcode Lookup','Description1','Price']]
df.loc[:,'Price'] = df.loc[:,'Price'] * 0.75
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Barcode Lookup  108 non-null    int64  
 1   Description1    108 non-null    object 
 2   Price           108 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.7+ KB


In [1097]:
doc = docx.Document()
style = doc.styles['Normal']
font = style.font
font.size = Pt(11)
font.name='Calibri'

In [1098]:
data = list_df(df)
for d in data:
    print(d)
    create_table(doc,d)
print('File is Ready.')

    Barcode Lookup              Description1   Price
0     195699578913  BARELY BAGGIES SHORT 2.5   44.25
1     195699192317             CAP MW BOTTOM   59.25
2     195699192942           CAP MW ZIP NECK   66.75
3     195699193031           CAP MW ZIP NECK   66.75
4     195699193130           CAP MW ZIP NECK   66.75
5     195699577404       LW SYNCH SNAP-T P/O   96.75
6     195699008892        R2 TECHFACE JACKET  134.25
7     195699673304         ALPINE GUIDE PANT  186.75
8     195699673359         ALPINE GUIDE PANT  186.75
9     195699673434         ALPINE GUIDE PANT  186.75
10    195699673496         ALPINE GUIDE PANT  186.75
11    195699673564         ALPINE GUIDE PANT  186.75
12    195699673649         ALPINE GUIDE PANT  186.75
13    195699903364  BARELY BAGGIES SHORT 2.5   44.25
14    195699903388  BARELY BAGGIES SHORT 2.5   44.25
15    195699903395  BARELY BAGGIES SHORT 2.5   44.25
16    195699903418  BARELY BAGGIES SHORT 2.5   44.25
17    195699903425  BARELY BAGGIES SHORT 2.5  

In [1099]:
set_section_margins(doc)

In [1100]:
doc.save('tags.docx')

In [1101]:
# Removes all images of barcodes created to save memory
filelist = glob.glob(os.path.join('Barcodes', "*.png"))
for f in filelist:
    os.remove(f)